<a href="https://colab.research.google.com/github/justadudewhohacks/ipynbs/blob/master/age_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies


In [0]:
!pip install -U -q PyDrive

In [0]:
!pip install git+https://github.com/justadudewhohacks/image_augment.py

## Download Data

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

train_data_json_id = '1CDMRQdAhcws_g1yDw_29ZD5DNDDyi7Xw'
test_data_json_id = '1_0dpT5HRTWocnK35KLQFDHzJiwV2-IQZ'

utk_images_7z_id = '1c61PoUhIPKeoRzB0XDI23XMDyJaCfKSh'
utk_landmarks_7z_id = '1Nxg7KKfEkDBWCqhusE1S6Edp6n3tTOuN'

appareal_labels_json_id = '1_zfGunGuqyrftDJIEKw6NVJOS55vyOrh'
appareal_images_7z_id = '1BDm6r88XLwDFsqOa2ZbbUtW1HDyHo5yA'
appareal_landmarks_7z_id = '1Am36Tk-BnjfV1d8_iUpRcW-cPfQtAN0H'

wiki_labels_json_id = '1BamAqN3tNEMh6kNQQ4C8nWf6gOA2IS6X'
wiki_images_7z_id = '1Fy3pi-Pra1IsN9HDD268nRvXa1TbsryE'
wiki_landmarks_7z_id = '1M-YeSGEEboVqNK8pTCJhbxeVaLp0TKJ4'

if not os.path.exists('./data'):
  os.makedirs('./data')
if not os.path.exists('./data/utk'):
  os.makedirs('./data/utk')
if not os.path.exists('./data/appareal'):
  os.makedirs('./data/appareal')
if not os.path.exists('./data/wiki'):
  os.makedirs('./data/wiki')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
    
print('downloading trainData.json and testData.json ...')
drive.CreateFile({ 'id': train_data_json_id }).GetContentFile('./data/trainData.json')
drive.CreateFile({ 'id': test_data_json_id }).GetContentFile('./data/testData.json')

print('downloading utk data ...')
drive.CreateFile({ 'id': utk_images_7z_id }).GetContentFile('./data/utk/images.7z')
drive.CreateFile({ 'id': utk_landmarks_7z_id }).GetContentFile('./data/utk/landmarks.7z')

print('downloading appareal data ...')
drive.CreateFile({ 'id': appareal_labels_json_id }).GetContentFile('./data/appareal/labels.json')
drive.CreateFile({ 'id': appareal_images_7z_id }).GetContentFile('./data/appareal/images.7z')
drive.CreateFile({ 'id': appareal_landmarks_7z_id }).GetContentFile('./data/appareal/landmarks.7z')

print('downloading wiki data ...')
drive.CreateFile({ 'id': wiki_labels_json_id }).GetContentFile('./data/wiki/labels.json')
drive.CreateFile({ 'id': wiki_images_7z_id }).GetContentFile('./data/wiki/images.7z')
drive.CreateFile({ 'id': wiki_landmarks_7z_id }).GetContentFile('./data/wiki/landmarks.7z')
  
print('unzipping data...')

!rm -rf ./sample_data
!cd ./data/utk && p7zip -d ./images.7z >> ../../utk-images.unzip.txt
!cd ./data/utk && p7zip -d ./landmarks.7z >> ../../utk-landmarks.unzip.txt
!cd ./data/appareal && p7zip -d ./images.7z >> ../../appareal-images.unzip.txt
!cd ./data/appareal && p7zip -d ./landmarks.7z >> ../../appareal-landmarks.unzip.txt
!cd ./data/wiki && p7zip -d ./images.7z >> ../../wiki-images.unzip.txt
!cd ./data/wiki && p7zip -d ./landmarks.7z >> ../../wiki-landmarks.unzip.txt
!rm -rf *.unzip.txt
print('done!')

## Training

### Imports

In [0]:
import cv2
import math
import json
import random
import time
import os
import numpy as np
import google.colab as colab
import tensorflow as tf

### Weight Serialization

In [0]:
class WeightInitializer:
  def __init__(self, weight_initializer, bias_initializer):
    self.weight_initializer = weight_initializer
    self.bias_initializer = bias_initializer
  
  def process_batch_norm_weights(self, channels, name):
    with tf.variable_scope(name):
      tf.get_variable('mean', shape = [channels], initializer = tf.keras.initializers.Zeros)
      tf.get_variable('variance', shape = [channels], initializer = tf.keras.initializers.Ones)
      tf.get_variable('offset', shape = [channels], initializer = tf.keras.initializers.Zeros)
      tf.get_variable('scale', shape = [channels], initializer = tf.keras.initializers.Ones)
    
  def process_conv_weights(self, channels_in, channels_out, name, filter_size = 3, with_batch_norm = False):
    with tf.variable_scope(name):
      self.weight_initializer('filter', [filter_size, filter_size, channels_in, channels_out])
      if with_batch_norm:
        self.process_batch_norm_weights(channels_out, 'batch_norm')
      else:
        self.bias_initializer('bias',[channels_out])
    
  def process_fc_weights(self, channels_in, channels_out, name):
    with tf.variable_scope(name):
      self.weight_initializer('weights', [channels_in, channels_out])
      self.bias_initializer('bias',[channels_out])
    
  def process_depthwise_separable_conv2d_weights(self, channels_in, channels_out, name, with_batch_norm = False):
    with tf.variable_scope(name):
      self.weight_initializer('depthwise_filter', [3, 3, channels_in, 1])
      self.weight_initializer('pointwise_filter', [1, 1, channels_in, channels_out])
      if with_batch_norm:
        self.process_batch_norm_weights(channels_out, 'batch_norm')
      else:
        self.bias_initializer('bias',[channels_out])

  def process_dense_block_weights(self, channels_in, channels_out, name, is_first_layer = False, use_depthwise_separable_conv2d = True, with_batch_norm = False):
    conv_weight_processor = self.process_depthwise_separable_conv2d_weights if use_depthwise_separable_conv2d else self.process_conv_weights
    conv0_weight_processor = self.process_conv_weights if is_first_layer else conv_weight_processor
    
    with tf.variable_scope(name):
      conv0_weight_processor(channels_in, channels_out, 'conv0', with_batch_norm = with_batch_norm)
      conv_weight_processor(channels_out, channels_out, 'conv1', with_batch_norm = with_batch_norm)
      conv_weight_processor(channels_out, channels_out, 'conv2', with_batch_norm = with_batch_norm)
      conv_weight_processor(channels_out, channels_out, 'conv3', with_batch_norm = with_batch_norm)

### Neural Network

In [0]:
def batch_norm(x, name):
  with tf.variable_scope(name):
    return tf.nn.batch_normalization(x, tf.get_variable('mean'), tf.get_variable('variance'), tf.get_variable('offset'), tf.get_variable('scale'), 1e-3)

def conv2d(x, name, stride, with_batch_norm = False):
  with tf.variable_scope(name):
    out = tf.nn.conv2d(x, tf.get_variable('filter'), stride, 'SAME')
    out = batch_norm(out, 'batch_norm') if with_batch_norm else tf.add(out, tf.get_variable('bias'))
    return out

def depthwise_separable_conv2d(x, name, stride, with_batch_norm = False):
  with tf.variable_scope(name):
    out = tf.nn.separable_conv2d(x, tf.get_variable('depthwise_filter'), tf.get_variable('pointwise_filter'), stride, 'SAME')
    out = batch_norm(out, weights['batch_norm']) if with_batch_norm else tf.add(out, tf.get_variable('bias'))
    return out
  
def fully_connected(x, name):
  with tf.variable_scope(name):
    weights = tf.get_variable('weights')
    out = tf.reshape(x, [-1, weights.get_shape().as_list()[0]])
    out = tf.matmul(out, weights)
    out = tf.add(out, tf.get_variable('bias'))
  return out

def dense_block(x, name, is_first_layer = False, is_scale_down = True, use_depthwise_separable_conv2d = True, with_batch_norm = False):
  conv_op = depthwise_separable_conv2d if use_depthwise_separable_conv2d else conv2d
  initial_stride = [1, 2, 2, 1]  if is_scale_down else [1, 1, 1, 1]
  
  with tf.variable_scope(name):
    if is_first_layer: 
      out1 = conv2d(x, 'conv0', initial_stride, with_batch_norm = with_batch_norm) 
    else: 
      out1 = conv_op(x, 'conv0', initial_stride, with_batch_norm = with_batch_norm)

    in2 = tf.nn.relu(out1)
    out2 = conv_op(in2, 'conv1', [1, 1, 1, 1], with_batch_norm = with_batch_norm)

    in3 = tf.nn.relu(tf.add(out1, out2))
    out3 = conv_op(in3, 'conv2', [1, 1, 1, 1], with_batch_norm = with_batch_norm)

    in4 = tf.nn.relu(tf.add(out1, tf.add(out2, out3)))
    out4 = conv_op(in4, 'conv3', [1, 1, 1, 1], with_batch_norm = with_batch_norm)

    return tf.nn.relu(tf.add(out1, tf.add(out2, tf.add(out3, out4))))

def normalize(x, mean_rgb):
  r, g, b = mean_rgb
  shape = np.append(np.array(x.shape[0:3]), [1])
  avg_r = tf.fill(shape, r)
  avg_g = tf.fill(shape, g)
  avg_b = tf.fill(shape, b)
  avg_rgb = tf.concat([avg_r, avg_g, avg_b], 3)

  return tf.divide(tf.subtract(x, avg_rgb), 255)


'''
--------------------------------------------------------------------------------

NeuralNetwork base class

--------------------------------------------------------------------------------
'''

class NeuralNetwork:
  def __init__(self, initialize_weights, name = None):
    self.initialize_weights = initialize_weights
    self.name = name
  
  def init_trainable_weights(self, weight_initializer = tf.keras.initializers.glorot_normal(), bias_initializer = tf.keras.initializers.Zeros()):
    def initialize_weights_factory(initializer):
      def initialize_weights(name, shape):
        return tf.get_variable(name, initializer = initializer(shape))
      
      return initialize_weights
      
    self.initialize_weights(WeightInitializer(initialize_weights_factory(weight_initializer), initialize_weights_factory(bias_initializer)))

  def load_weights(self, checkpoint_file, weight_initializer = tf.keras.initializers.glorot_normal(), bias_initializer = tf.keras.initializers.Zeros()):
    checkpoint_data = np.load(checkpoint_file + '.npy')
    meta_json = load_json(checkpoint_file + '.json')

    idx = 0
    data_idx = 0
    
    def initialize_weights_factory(initializer):
      def initialize_weights(name, shape):
        nonlocal idx, data_idx
        if (idx >= len(meta_json)):
          print('load_weights - warning meta_json does not contain data for idx: ' + str(idx) + ', using default initializer')
          return tf.get_variable(name, initializer = initializer(shape))

        size = 1
        for val in shape:
          size = size * val
        initial_value = np.reshape(checkpoint_data[data_idx:data_idx + size], shape)
        
        var = tf.get_variable(name, initializer = initial_value.astype(np.float32))
     
        if (var.shape != meta_json[idx]['shape']):
          raise Exception('load_weights - shapes not matching at variable ' + str(var.name) + ': ' + str(var.shape) + ' and ' + str(meta_json[idx]['shape']))
           
        if (var.name != meta_json[idx]['name']):
          print('load_weights - warning: variable names not matching: ' + str(var.name) + ' and ' + str(meta_json[idx]['name']))
        
        idx += 1
        data_idx += size

        return var
      
      return initialize_weights
    
    self.initialize_weights(WeightInitializer(initialize_weights_factory(weight_initializer), initialize_weights_factory(bias_initializer)))

      
'''
--------------------------------------------------------------------------------

Neural Networks

--------------------------------------------------------------------------------
'''
      
class Densenet_4_4_FeatureExtractor(NeuralNetwork):
  def __init__(self, use_depthwise_separable_conv2d = True, with_batch_norm = False, name = 'densenet_4_4_feature_extractor'):
    super().__init__(self.initialize_weights, name = name)
    self.use_depthwise_separable_conv2d = use_depthwise_separable_conv2d
    self.with_batch_norm = with_batch_norm
  
  def _create_dense_block_weight_processor(self, weight_processor):
    def _dense_block_weight_processor(cin, cout, name, is_first_layer = False):
      return weight_processor.process_dense_block_weights(
        cin, 
        cout, 
        name, 
        is_first_layer = is_first_layer, 
        use_depthwise_separable_conv2d = self.use_depthwise_separable_conv2d, 
        with_batch_norm = self.with_batch_norm
      )
    
    return _dense_block_weight_processor
  
  def _create_dense_block(self):
    def _dense_block(x, name, is_first_layer = False):
      return dense_block(
        x, 
        name, 
        is_first_layer = is_first_layer, 
        use_depthwise_separable_conv2d = self.use_depthwise_separable_conv2d, 
        with_batch_norm = self.with_batch_norm
      )
    
    return _dense_block
  
  def initialize_weights(self, weight_processor, variable_scope = 'feature_extractor'):
    process_dense_block_weights = self._create_dense_block_weight_processor(weight_processor)
    
    with tf.variable_scope(self.name):
      with tf.variable_scope(variable_scope):
        process_dense_block_weights(3, 32, 'dense0', is_first_layer = True)
        process_dense_block_weights(32, 64, 'dense1')
        process_dense_block_weights(64, 128, 'dense2')
        process_dense_block_weights(128, 256, 'dense3')
    
  def forward(self, batch_tensor, variable_scope = 'feature_extractor'):
    dense_block = self._create_dense_block()
    
    mean_rgb = [122.782, 117.001, 104.298]
    normalized = normalize(batch_tensor, mean_rgb)
    
    with tf.variable_scope(self.name, reuse = True):
      with tf.variable_scope(variable_scope):
        out = dense_block(normalized, 'dense0', is_first_layer = True)
        out = dense_block(out, 'dense1')
        out = dense_block(out, 'dense2')
        out = dense_block(out, 'dense3')

      return out  
  
class Densenet_4_5_FeatureExtractor(Densenet_4_4_FeatureExtractor):
  def __init__(self, use_depthwise_separable_conv2d = True, with_batch_norm = False, name = 'densenet_4_5_feature_extractor'):
    super().__init__(
      use_depthwise_separable_conv2d = use_depthwise_separable_conv2d,
      with_batch_norm = with_batch_norm,
      name = name
    )
    
  def initialize_weights(self, weight_processor, variable_scope = 'feature_extractor'):
    process_dense_block_weights = super()._create_dense_block_weight_processor(weight_processor)
    
    super().initialize_weights(weight_processor, variable_scope = variable_scope)
    with tf.variable_scope(self.name):
      with tf.variable_scope(variable_scope):
        process_dense_block_weights(256, 512, 'dense4')
    
  def forward(self, batch_tensor, variable_scope = 'feature_extractor'):
    dense_block = super()._create_dense_block()
    
    out = super().forward(batch_tensor, variable_scope = variable_scope)
    with tf.variable_scope(self.name, reuse = True):
      with tf.variable_scope(variable_scope):
        out = dense_block(out, 'dense4')

      return out
  
class DenseMobilenet_4_4(Densenet_4_4_FeatureExtractor):
  def __init__(self, use_depthwise_separable_conv2d = True, with_batch_norm = False, name = 'densenet_4_4'):
    super().__init__(
      use_depthwise_separable_conv2d = use_depthwise_separable_conv2d,
      with_batch_norm = with_batch_norm, 
      name = name
    )
  
  def initialize_weights(self, weight_processor):
    super().initialize_weights(weight_processor)
    
    with tf.variable_scope(self.name):
      with tf.variable_scope('classifier'):
        weight_processor.process_fc_weights(256, 1, 'fc_age')
    
  def forward(self, batch_tensor):
    out = super().forward(batch_tensor)
    with tf.variable_scope(self.name, reuse = True):
      with tf.variable_scope('classifier'):
        out = tf.nn.avg_pool(out, [1, 7, 7, 1], [1, 2, 2, 1], 'VALID')
        out = fully_connected(out, 'fc_age')

    return out
  
class DenseMobilenet_4_5(Densenet_4_5_FeatureExtractor):
  def __init__(self, use_depthwise_separable_conv2d = True, with_batch_norm = False, name = 'densenet_4_5'):
    super().__init__(
      use_depthwise_separable_conv2d = use_depthwise_separable_conv2d,
      with_batch_norm = with_batch_norm,
      name = name
    )
    
  def initialize_weights(self, weight_processor):
    super().initialize_weights(weight_processor)
    
    with tf.variable_scope(self.name):
      with tf.variable_scope('classifier'):
        weight_processor.process_fc_weights(512, 1, 'fc_age')
    
  def forward(self, batch_tensor):
    out = super().forward(batch_tensor)
    
    with tf.variable_scope(self.name, reuse = True):
      with tf.variable_scope('classifier'):
        out = tf.nn.avg_pool(out, [1, 7, 7, 1], [1, 2, 2, 1], 'VALID')
        out = fully_connected(out, 'fc_age')
    
    return out
  
class DenseMobilenet_4_4_DEX(Densenet_4_4_FeatureExtractor):
  def __init__(self, use_depthwise_separable_conv2d = True, with_batch_norm = False, name = 'densenet_4_4_dex'):
    super().__init__(
      use_depthwise_separable_conv2d = use_depthwise_separable_conv2d,
      with_batch_norm = with_batch_norm,
      name = name
    )
  
  def initialize_weights(self, weight_processor):
    super().initialize_weights(weight_processor)
    with tf.variable_scope(self.name):
      with tf.variable_scope('classifier'):
        weight_processor.process_fc_weights(256, 101, 'fc_age')
    
  def forward(self, batch_tensor):
    out = super().forward(batch_tensor)
    with tf.variable_scope(self.name, reuse = True):
      with tf.variable_scope('classifier'):
        out = tf.nn.avg_pool(out, [1, 7, 7, 1], [1, 2, 2, 1], 'VALID')
        out = fully_connected(out, 'fc_age')
        out = tf.reduce_sum(tf.multiply(tf.nn.softmax(out), np.arange(101)), axis = 1)
    
    return out


### Preprocessing

In [0]:
def num_in_range(val, min_val, max_val):
  return min(max(min_val, val), max_val)

def apply_intensity_adjustment(img, params):
  has_alpha = 'alpha' in params
  has_beta = 'beta' in params

  if not has_alpha and not has_beta:
    print('warning: intensity dict neither has alpha nor beta')

  alpha = params['alpha'] if has_alpha else 1.0
  beta = params['beta'] if has_alpha else 0.0

  return np.clip((img * alpha + beta), 0, 255).astype(img.dtype)

def apply_to_gray(img):
  return cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.COLOR_GRAY2BGR)

def apply_blur(img, params):
  has_kernel_size = 'kernel_size' in params
  has_std_dev = 'std_dev' in params

  if not has_kernel_size and not has_std_dev:
    print('warning: blur dict neither has kernel_size nor std_dev')

  kernel_size = params['kernel_size'] if has_kernel_size else 3
  std_dev = params['std_dev'] if has_std_dev else 1.0

  return cv2.GaussianBlur(img, (kernel_size, kernel_size), std_dev, std_dev)

def apply_random_crop(img, bbox):
  height, width = img.shape[:2]

  min_x, min_y, max_x, max_y = bbox
  min_x = int(num_in_range(min_x, 0, 1) * width)
  min_y = int(num_in_range(min_y, 0, 1) * height)
  max_x = int(num_in_range(max_x, 0, 1) * width)
  max_y = int(num_in_range(max_y, 0, 1) * height)
  x0 = random.randint(0, min_x)
  y0 = random.randint(0, min_y)
  x1 = random.randint(0, abs(width - max_x)) + max_x
  y1 = random.randint(0, abs(height - max_y)) + max_y

  return img[y0:y1, x0:x1]

def apply_rotate(img, angle):
  height, width = img.shape[:2]
  cx, cy = int(width / 2), int(height / 2)

  M = cv2.getRotationMatrix2D((cx, cy), -angle, 1.0)
  cos = np.abs(M[0, 0])
  sin = np.abs(M[0, 1])

  new_width = int((height * sin) + (width * cos))
  new_height = int((height * cos) + (width * sin))

  M[0, 2] += (new_width / 2) - cx
  M[1, 2] += (new_height / 2) - cy

  return cv2.warpAffine(img, M, (new_width, new_height))

def augment(
  img,
  intensity = None, # (alpha, beta)
  blur = None, # (kernel_size, stddev)
  to_gray = False,
  random_crop = None, # [(x0, y0, x1, y1)]
  flip = False,
  rotation_angle = None
):
  img = apply_intensity_adjustment(img, intensity) if intensity is not None else img
  img = apply_blur(img, blur) if blur is not None else img
  img = apply_to_gray(img) if to_gray is True else img

  # TODO noise

  img = apply_random_crop(img, random_crop) if random_crop is not None else img
  # TODO shearing
  img = cv2.flip(img, 1) if flip is True else img
  img = apply_rotate(img, rotation_angle) if rotation_angle is not None else img

  return img

def resize_preserve_aspect_ratio(img, size):
  height, width = img.shape[:2]
  max_dim = max(height, width)
  ratio = size / float(max_dim)
  shape = (height * ratio, width * ratio)
  resized_img = cv2.resize(img, (int(round(width * ratio)), int(round(height * ratio))))

  return resized_img

def pad_to_square(img):
  if len(img.shape) == 2:
    img = np.expand_dims(img, axis = 2)

  height, width, channels = img.shape
  max_dim = max(height, width)
  square_img = np.zeros([max_dim, max_dim, channels], dtype = img.dtype)

  dx = math.floor(abs(max_dim - width) / 2)
  dy = math.floor(abs(max_dim - height) / 2)
  square_img[dy:dy + height,dx:dx + width] = img

  return square_img

### Batch Loader

In [0]:
def load_json(json_file_path):
  with open(json_file_path) as json_file:  
    return json.load(json_file)

def shuffle_array(arr):
  arr_clone = arr[:]
  random.shuffle(arr_clone)
  return arr_clone
     
class BatchLoader:
  def __init__(
    self, 
    data,
    resolve_image_path,
    extract_data_labels,
    augment_image = None, 
    is_test = False,
    start_epoch = None
  ):
    if not is_test and start_epoch == None:
      raise Exception('DataLoader - start_epoch has to be defined in train mode')
    
    self.data = data
    self.resolve_image_path = resolve_image_path
    self.extract_data_labels = extract_data_labels
    self.augment_image = augment_image
    self.is_test = is_test
    self.epoch = start_epoch
    self.buffered_data = shuffle_array(self.data) if not is_test else self.data
    self.current_idx = 0
 
  def get_end_idx(self):
    return len(self.buffered_data)
   
  def load_image(self, data, image_size):
    db = data['db']
    img_file = data['file']
    file_suffix = 'chip_0' if db == 'utk' else ('face_0' if db == 'appareal' else '')
    landmarks_file = img_file.replace(file_suffix + '.jpg', file_suffix + '.json')
    landmarks_file_path = './data/' + db + '/landmarks/' + landmarks_file

    img = cv2.imread(self.resolve_image_path(data))
    if img is None:
      raise Exception('failed to read image from path: ' + img_file_path)

    if (self.augment_image is not None):
      img = self.augment_image(img, data)

    img = pad_to_square(resize_preserve_aspect_ratio(img, image_size))

    return img

  def load_image_batch(self, datas, image_size):
    preprocessed_imgs = []
    for data in datas:
      preprocessed_imgs.append(self.load_image(data, image_size))
    return np.stack(preprocessed_imgs, axis = 0)
    
  def load_labels(self, datas):
    labels = []
    for data in datas:
      labels.append(self.extract_data_labels(data))
    return np.stack(labels, axis = 0)
    
  def next_batch(self, batch_size, image_size = 112):
    if batch_size < 1:
      raise Exception('DataLoader.next_batch - invalid batch_size: ' + str(batch_size))
      
    
    from_idx = self.current_idx
    to_idx = self.current_idx + batch_size
    
    # end of epoch
    if (to_idx > len(self.buffered_data)):
      if self.is_test:
        to_idx = len(self.buffered_data)
        if to_idx == self.current_idx:
          return None
      else:
        self.epoch += 1
        self.buffered_data = self.buffered_data[from_idx:] + shuffle_array(self.data)  
        from_idx = 0
        to_idx = batch_size
      
    self.current_idx = to_idx
    
    next_data = self.buffered_data[from_idx:to_idx]
      
    batch_x = self.load_image_batch(next_data, image_size)
    batch_y = self.load_labels(next_data)
    
    return batch_x, batch_y

### Common

In [0]:
'''
--------------------------------------------------------------------------------

utility

--------------------------------------------------------------------------------
'''

def gpu_session(callback):
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  config.allow_soft_placement = True
  config.log_device_placement = True
  with tf.Session(config = config) as session:
    with tf.device('/gpu:0'):
      callback(session)

def get_checkpoint(epoch):
  return model_name + '.ckpt-' + str(epoch)

def download_epoch_files(start, end):
  for epoch in range(start, end):
    colab.files.download('epoch_' + str(epoch) + '.txt')
    colab.files.download(get_checkpoint(epoch) + '.index') 
    colab.files.download(get_checkpoint(epoch) + '.meta') 
    colab.files.download(get_checkpoint(epoch) + '.data-00000-of-00001')

def save_weights(var_list, checkpoint_file):
  checkpoint_data = np.array([])
  meta_data = []
  for var in var_list:
    meta_data.append({ 'shape': var.get_shape().as_list(), 'name': var.name })
    checkpoint_data = np.append(checkpoint_data, var.eval().flatten())
    
  meta_json = open(checkpoint_file + '.json', 'w')
  meta_json.write(json.dumps(meta_data))
  meta_json.close()
  np.save(checkpoint_file, checkpoint_data)


'''
--------------------------------------------------------------------------------

Data Loader

--------------------------------------------------------------------------------
'''


appareal_labels = load_json('./data/appareal/labels.json')
wiki_labels = load_json('./data/wiki/labels.json')

def extract_data_labels(data):
  db = data['db']
  img_file = data['file']

  if db == 'utk':
    age = int(float(img_file.split('_')[0]))
    return age
  elif db == 'appareal':
    age = appareal_labels[img_file]['age']
    return age
  elif db == 'wiki':
    age = wiki_labels[img_file]['age']
    return age
  else: raise('unknown db: ' + db)
    
def resolve_image_path(data):
  db = data['db']
  img_file = data['file']
  return './data/' + db + '/cropped-images/' + img_file   

def min_bbox(landmarks):
  min_x, min_y, max_x, max_y = 1.0, 1.0, 0, 0
  for pt in landmarks:
    min_x = pt['x'] if pt['x'] < min_x else min_x
    min_y = pt['y'] if pt['y'] < min_y else min_y
    max_x = max_x if pt['x'] < max_x else pt['x']
    max_y = max_y if pt['y'] < max_y else pt['y']

  return [min_x, min_y, max_x, max_y]

def augment_image(img, data):
  db = data['db']
  img_file = data['file']
  file_suffix = 'chip_0' if db == 'utk' else ('face_0' if db == 'appareal' else '')
  landmarks_file = img_file.replace(file_suffix + '.jpg', file_suffix + '.json')
  landmarks_file_path = './data/' + db + '/landmarks/' + landmarks_file

  landmarks = load_json(landmarks_file_path)
  return augment(
    img,
    random_crop = min_bbox(landmarks),
    flip = random.random() < 0.5,
    rotation_angle = random.uniform(-15, 15),
    blur = { 'kernel_size':  random.choice([3, 5, 7, 9, 11]), 'std_dev': random.uniform(0.5, 1.5) },
    intensity = { 'alpha': random.uniform(0.5, 1.5), 'beta': random.uniform(-20, 20) },
    to_gray = random.random() < 0.2
    #rotation_angle = random.uniform(-5, 5),
    #blur = { 'kernel_size':  random.choice([3, 5, 7]), 'std_dev': random.uniform(0.8, 1.2) },
    #intensity = { 'alpha': random.uniform(0.8, 1.2), 'beta': random.uniform(-10, 10) },
    #to_gray = random.random() < 0.1
  )

class DataLoader(BatchLoader):
  def __init__(self, data, with_augmentation = False, start_epoch = None, is_test = False):
    BatchLoader.__init__(
      self, 
      data, 
      resolve_image_path, 
      extract_data_labels, 
      augment_image = augment_image if with_augmentation else None, 
      start_epoch = start_epoch, 
      is_test = is_test
    )

### Training

In [0]:
tf.reset_default_graph()

net = DenseMobilenet_4_4_DEX()
model_name = './dense_mobilenet_4_4_dex'
image_size = 112

# training parameters
learning_rate = 0.0001
start_epoch = 1
end_epoch = 2000
batch_size = 32

#optimizer = tf.train.GradientDescentOptimizer(learning_rate)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

X = tf.placeholder(tf.float32, [batch_size, 112, 112, 3])
Y = tf.placeholder(tf.float32, [batch_size])

train_data = load_json('./data/trainData.json')
data_loader = DataLoader(train_data, start_epoch = start_epoch, with_augmentation = True)
net.init_trainable_weights()

age = net.forward(X)
loss_op = tf.losses.absolute_difference(age, Y)
train_op = optimizer.minimize(loss_op)

log_file = open('./log.txt', 'w')

saver = tf.train.Saver(max_to_keep = None)
  
def train(sess):
  total_loss = 0
  iteration_count = 0
  ts_epoch = time.time()
  
  sess.run(tf.global_variables_initializer())

  if (start_epoch != 0):
    checkpoint = get_checkpoint(start_epoch - 1)
    saver.restore(sess, checkpoint)
    print('done restoring session')

  while data_loader.epoch <= end_epoch:
    epoch = data_loader.epoch
    current_idx = data_loader.current_idx
    end_idx = data_loader.get_end_idx()

    ts = time.time()

    batch_x, batch_y = data_loader.next_batch(batch_size, image_size)

    loss, _ = sess.run([loss_op, train_op], feed_dict = { X: batch_x, Y: batch_y })
    total_loss += loss
    iteration_count += 1
    log_file.write("epoch " + str(epoch) + ", (" + str(current_idx) + " of " + str(end_idx) + "), loss= " + "{:.4f}".format(loss) 
          + ", time= " + str((time.time() - ts) * 1000) + "ms \n")

    if epoch != data_loader.epoch:
      print('next epoch: ' + str(data_loader.epoch))
      print('avg_loss= ' + str(total_loss / iteration_count))
      saver.save(sess, model_name + '.ckpt', global_step = epoch)

      epoch_txt_file_path = 'epoch_' + str(epoch) + '.txt'
      epoch_txt = open(epoch_txt_file_path, 'w')
      epoch_txt.write('total_loss= ' + str(total_loss) + '\n')
      epoch_txt.write('avg_loss= ' + str(total_loss / iteration_count) + '\n')
      epoch_txt.write('learning_rate= ' + str(learning_rate) + '\n')
      epoch_txt.write('batch_size= ' + str(batch_size) + '\n')
      epoch_txt.write('epoch_time= ' + str(time.time() - ts_epoch) + 's \n')
      epoch_txt.close()

      #colab.files.download(epoch_txt_file_path)
      #colab.files.download(get_checkpoint(epoch) + '.index') 
      #colab.files.download(get_checkpoint(epoch) + '.meta') 
      #colab.files.download(get_checkpoint(epoch) + '.data-00000-of-00001')

      total_loss = 0
      iteration_count = 0              
      ts_epoch = time.time()
        
  print('done!')
  log_file.close() 
    
gpu_session(train)

INFO:tensorflow:Restoring parameters from ./dense_mobilenet_4_4_dex.ckpt-0
done restoring session


### Testing

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = True

net = DenseMobilenet_4_4_DEX()
model_name = './dense_mobilenet_4_4_dex'

def get_checkpoint(epoch):
  return model_name + '.ckpt-' + str(epoch)

def compile_loss_op(X, Y, weight_map):
  age = net.forward(X, weight_map.weights)
  loss_op = tf.losses.absolute_difference(age, Y)
  return loss_op

batch_size = 32
dbs = ['utk', 'wiki', 'appareal']
test_data = load_json('./data/testData.json')

for epoch in range(37, -1, -1):
  tf.reset_default_graph()
  weight_map = net.init_trainable_weights()

  X = tf.placeholder(tf.float32, [batch_size, 112, 112, 3])
  Y = tf.placeholder(tf.float32, [batch_size])
  loss_op = compile_loss_op(X, Y, weight_map)

  test_txt = open('test_epoch_' + str(epoch) + '.txt', 'w')

  init = tf.global_variables_initializer()
  saver = tf.train.Saver(max_to_keep = None)

  total_loss = 0
  iteration_count = 0
  ts_test = time.time()
  #with tf.Session(tpu_address) as sess:
  with tf.Session(config = config) as sess:
    checkpoint = get_checkpoint(epoch)
    sess.run(init)
    saver.restore(sess, checkpoint)
    
    with tf.device('/gpu:0'):

      total_loss_db = 0
      iteration_count_db = 0
      for db in dbs:
        db_data = []
        for data in test_data:
          if data['db'] == db:
            db_data.append(data)

        data_loader = DataLoader(db_data, is_test = True)
        next_batch = data_loader.next_batch(batch_size)
        while next_batch != None:
          batch_x, batch_y = next_batch
          if batch_x.shape[0] != batch_size:
            X_tmp = tf.placeholder(tf.float32, [batch_x.shape[0], 112, 112, 3])
            Y_tmp = tf.placeholder(tf.float32, [batch_x.shape[0]])
            loss_op_tmp = compile_loss_op(X_tmp, Y_tmp, weight_map)
            loss = sess.run(loss_op_tmp, feed_dict = { X_tmp: batch_x, Y_tmp: batch_y }) #/ batch_x.shape[0]
          else:
            loss = sess.run(loss_op, feed_dict = { X: batch_x, Y: batch_y }) #/ batch_size
          total_loss += loss
          total_loss_db += loss
          iteration_count += 1
          iteration_count_db += 1
          next_batch = data_loader.next_batch(batch_size)

        print(str(db) + ", avg_loss= " + str(total_loss_db / iteration_count_db))
        test_txt.write(str(db) + ":" + '\n')
        test_txt.write('total_loss= ' + str(total_loss_db) + '\n')
        test_txt.write('avg_loss= ' + str(total_loss_db / iteration_count_db) + '\n')
        test_txt.write('\n')
        total_loss_db = 0
        iteration_count_db = 0

      print("avg_loss= " + str(total_loss / iteration_count))
      test_txt.write('----------------\n\n')
      test_txt.write('total_loss= ' + str(total_loss) + '\n')
      test_txt.write('avg_loss= ' + str(total_loss / iteration_count) + '\n')
      test_txt.write('test_time= ' + str(time.time() - ts_test) + 's \n')
      test_txt.close()

### Overfit

In [0]:
tf.reset_default_graph()

net = DenseMobilenet_4_4_DEX()
model_name = './dense_mobilenet_4_4_dex'
image_size = 112

# training parameters
learning_rate = 0.0001
start_epoch = 0
end_epoch = 2000
batch_size = 16

#optimizer = tf.train.GradientDescentOptimizer(learning_rate)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

X = tf.placeholder(tf.float32, [batch_size, 112, 112, 3])
Y = tf.placeholder(tf.float32, [batch_size])

train_data = load_json('./data/trainData.json')[0:batch_size]
data_loader = DataLoader(train_data, start_epoch = start_epoch)
net.init_trainable_weights()

age = net.forward(X)
loss_op = tf.losses.absolute_difference(age, Y)
train_op = optimizer.minimize(loss_op)

log_file = open('./log.txt', 'w')

saver = tf.train.Saver(max_to_keep = None)

def overfit(sess):
  total_loss = 0
  iteration_count = 0
  ts_epoch = time.time()

  sess.run(tf.global_variables_initializer())
  
  #saver.save(sess, model_name + '.ckpt', global_step = 0)
  if (start_epoch != 0):
    checkpoint = get_checkpoint(start_epoch - 1)
    saver.restore(sess, checkpoint)
    print('done restoring session')
 
  while data_loader.epoch <= end_epoch:
    epoch = data_loader.epoch
    current_idx = data_loader.current_idx
    end_idx = data_loader.get_end_idx()

    ts = time.time()

    batch_x, batch_y = data_loader.next_batch(batch_size, image_size)
    loss, _ = sess.run([loss_op, train_op], feed_dict = { X: batch_x, Y: batch_y })
    total_loss += loss
    iteration_count += 1
    log_file.write("epoch " + str(epoch) + ", (" + str(current_idx) + " of " + str(end_idx) + "), loss= " + "{:.4f}".format(loss) 
          + ", time= " + str((time.time() - ts) * 1000) + "ms \n")

    if epoch != data_loader.epoch:
      print(str(data_loader.epoch) + ': ' + "{:.4f}".format(total_loss / iteration_count) + ", " + "{:.4f}".format(total_loss))
      if False:
        print('next epoch: ' + str(data_loader.epoch))
        saver.save(sess, model_name + '.ckpt', global_step = epoch)

        epoch_txt_file_path = 'epoch_' + str(epoch) + '.txt'
        epoch_txt = open(epoch_txt_file_path, 'w')
        epoch_txt.write('total_loss= ' + str(total_loss) + '\n')
        epoch_txt.write('avg_loss= ' + str(total_loss / iteration_count) + '\n')
        epoch_txt.write('learning_rate= ' + str(learning_rate) + '\n')
        epoch_txt.write('batch_size= ' + str(batch_size) + '\n')
        epoch_txt.write('epoch_time= ' + str(time.time() - ts_epoch) + 's \n')
        epoch_txt.close()

      #colab.files.download(epoch_txt_file_path)
      #colab.files.download(get_checkpoint(epoch) + '.index') 
      #colab.files.download(get_checkpoint(epoch) + '.meta') 
      #colab.files.download(get_checkpoint(epoch) + '.data-00000-of-00001')

      total_loss = 0
      iteration_count = 0              
      ts_epoch = time.time()

  print('done!')
  log_file.close() 
  
gpu_session(overfit)

## Transfer Weights

In [0]:
tf.reset_default_graph()

net = DenseMobilenet_4_4_DEX()
net.load_weights('./feature_extractor')

optimizer = tf.train.AdamOptimizer(learning_rate = 100)

X = tf.placeholder(tf.float32, [batch_size, 112, 112, 3])
Y = tf.placeholder(tf.float32, [batch_size])

age = net.forward(X)
loss_op = tf.losses.absolute_difference(age, Y)
train_op = optimizer.minimize(loss_op)
print(len(tf.global_variables()))

saver = tf.train.Saver(max_to_keep = None)

def run1(sess):
  sess.run(tf.global_variables_initializer())
  saver.save(sess, 'dense_mobilenet_4_4_dex.ckpt-0')

gpu_session(run1)


In [0]:
tf.reset_default_graph()

src_net = DenseMobilenet_4_4()
src_net.load_weights('./feature_extractor')

vars1 = []

def run1(sess):
  sess.run(tf.global_variables_initializer())
  for var in tf.global_variables():
    vars1.append(var.eval())

gpu_session(run1)

tf.reset_default_graph()

net = DenseMobilenet_4_4_DEX()
net.init_trainable_weights()
saver = tf.train.Saver(max_to_keep = None)

vars2 = []

def run2(sess):
  sess.run(tf.global_variables_initializer())
  saver.restore(sess, 'dense_mobilenet_4_4_dex.ckpt-0')
  for var in tf.global_variables():
    vars2.append(var.eval())
  
gpu_session(run2) 

tf.reset_default_graph()

net = DenseMobilenet_4_4_DEX()
net.init_trainable_weights()
saver = tf.train.Saver(max_to_keep = None)

vars3 = []

def run3(sess):
  sess.run(tf.global_variables_initializer())
  saver.restore(sess, 'dense_mobilenet_4_4_dex.ckpt-0')
  for var in tf.global_variables():
    vars3.append(var.eval())
  
gpu_session(run3) 

print(np.array(vars1[0]) - np.array(vars3[0]))

